In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# ===== 2. Imports & hyperparameters =====
import math
import os
import re
import time
import torch
import torch.nn as nn
from torch.nn import functional as F
import pandas as pd

# ---------- Hyperparameters (tweakable) ----------
data_path = "/content/gdrive/MyDrive/japanese_texts.csv"  # update if needed
text_column_name = "text"  # name of column in CSV containing raw text

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device:", device)

# Model / training hyperparameters chosen for a longer training run
batch_size = 64          # sequences processed in parallel
block_size = 32         # context length for predictions
max_iters = 8000         # number of optimization steps (20+ minutes depends on GPU & data)
eval_interval = 500
eval_iters = 200
learning_rate = 3e-4

n_embd = 384             # embedding dimension
n_head = 6               # number of attention heads
n_layer = 6              # number of transformer blocks
dropout = 0.2

# Create output folder
os.makedirs("/content/model_checkpoints", exist_ok=True)

# ===== 3. Load CSV and combine text =====
print("Loading data from:", data_path)
df = pd.read_csv(data_path)
if text_column_name not in df.columns:
    raise ValueError(f"CSV must contain a column named '{text_column_name}'")

combined_text = " ".join(df[text_column_name].astype(str).tolist())
print("Loaded text length:", len(combined_text))

# ===== 4. Clean text: keep Japanese characters + some punctuation =====

def keep_japanese_characters(txt):
    allowed_ascii = set('!\"#%&\'()*+,-./0123456789:;<=>?@[]^_`{|}~ ')
    # Japanese punctuation: fullwidth punctuation '、。'
    # Extended: long vowel mark 'ー' U+30FC, iteration mark '々'
    chars = []
    for ch in txt:
        if ('\u3040' <= ch <= '\u309F') or ('\u30A0' <= ch <= '\u30FF') or ('\u4E00' <= ch <= '\u9FFF'):
            chars.append(ch)
        elif ch in allowed_ascii or ch in {'、', '。', '・', 'ー', '々', '「', '」', '『', '』', '〜'}:
            chars.append(ch)
        # else drop the character (emoji, latin letters, etc)
    return ''.join(chars)

clean_text = keep_japanese_characters(combined_text)
print("Length after cleaning:", len(clean_text))

# ===== 5. Build character vocabulary (stoi/itos) =====
chars = sorted(list(set(clean_text)))
vocab_size = len(chars)
print("Vocab size (unique characters):", vocab_size)
# For reproducibility, show first 100 chars (useful for debugging)
print("Sample chars:", ''.join(chars[:100]))

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

def encode(s):
    return [stoi[c] for c in s]

def decode(idx_list):
    return ''.join([itos[i] for i in idx_list])

# Test encode/decode
#test_str = "日本語のテキストを処理します。"
#enc = encode(test_str)
#dec = decode(enc)
#print("Test encode/decode OK:", dec == test_str)

# ===== 6. Convert to torch tensor and train/val split =====
data = torch.tensor(encode(clean_text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]
print("Train length:", len(train_data), "Val length:", len(val_data))

# ===== 7. Batching helper =====
import random
torch.manual_seed(1337)
def get_batch(split):
    data_split = train_data if split == 'train' else val_data
    ix = torch.randint(len(data_split) - block_size, (batch_size,))
    x = torch.stack([data_split[i:i+block_size] for i in ix])
    y = torch.stack([data_split[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

# Quick sanity check of batching:
xb, yb = get_batch('train')
print("xb shape:", xb.shape, "yb shape:", yb.shape)

# ===== 8. Build the Transformer (GPT-like) model =====
# Implementation based on minimal GPT-style architecture (Karpathy tutorial style)

class Head(nn.Module):
    """ Single self-attention head """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)    # (B, T, hs)
        q = self.query(x)  # (B, T, hs)
        wei = q @ k.transpose(-2, -1) * (k.shape[-1] ** -0.5)  # (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)  # (B, T, hs)
        out = wei @ v      # (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)            # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T, C)
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)    # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens):
        # idx is (B, T)
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]   # (B, vocab_size)
            probs = F.softmax(logits, dim=-1)
            next_idx = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, next_idx), dim=1)
        return idx

# Instantiate model
model = GPTLanguageModel().to(device)
print("Number of parameters (M):", sum(p.numel() for p in model.parameters()) / 1e6)

# ===== 9. Training helpers (loss estimate) =====
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean().item()
    model.train()
    return out

# ===== 10. Training loop =====
start_time = time.time()
model.train()
for iter in range(max_iters):
    # evaluate
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        elapsed = time.time() - start_time
        print(f"iter {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f} (elapsed {int(elapsed)}s)")
        # Save a checkpoint
        ckpt_path = f"/content/model_checkpoints/gpt_jp_iter{iter}.pt"
        torch.save({'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'iter': iter,
                    'stoi': stoi,
                    'itos': itos}, ckpt_path)

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # stabilize training
    optimizer.step()

# ===== 11. Generation demo =====
model.eval()
context = torch.zeros((1, 1), dtype=torch.long, device=device)  # start with "empty" token
generated = model.generate(context, max_new_tokens=200)[0].tolist()
print("=== Generated text ===")
print(decode(generated))

# ===== 12. Save final model =====
final_ckpt = "/content/model_checkpoints/gpt_jp_final.pt"
torch.save({'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'stoi': stoi,
            'itos': itos}, final_ckpt)
print("Saved final checkpoint to:", final_ckpt)

# =====================================================================
# End of notebook script
# =====================================================================


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Device: cpu
Loading data from: /content/gdrive/MyDrive/japanese_texts.csv
Loaded text length: 306
Length after cleaning: 306
Vocab size (unique characters): 135
Sample chars:  、。あいえかがきくこしすせただっつてでとなにねのはひべまめもらりるれわをんカグセタッデトナニプミメモラルレロンー一世予京人今会使処分力勉動化友味問変大天字定将工市強思成技文新方旅日明映昨最本来東毎気
Train length: 275 Val length: 31
xb shape: torch.Size([64, 256]) yb shape: torch.Size([64, 256])
Number of parameters (M): 10.842759


RuntimeError: random_ expects 'from' to be less than 'to', but got from=0 >= to=-225